**Imports**  
   Load the necessary libraries for data handling and date operations


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime


**Constants & File Paths**  
   Define the contract start/end dates and dynamically locate input CSVs under `data/`.


In [10]:
# Contract window
CSD, CED = "2019-07-20", "2019-10-10"

# Data files
base_path      = Path().resolve()
mprns_path     = base_path / "data" / "mprns.csv"
map_costs_path = base_path / "data" / "map_costs.csv"


**Helper: `load_and_preprocess_mprns`**
   - Read the MPRN metadata.  
   - Normalize column names to `snake_case`.  
   - Uppercase key ID fields for consistency.  
   - Capitalize `meter_type` to match the cost schedule.


In [3]:
def load_and_preprocess_mprns(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # snake_case columns
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    # uppercase ID fields
    for col in ["ldz", "exit_zone", "market_sector_code", "previous_status"]:
        if col in df.columns:
            df[col] = df[col].str.upper()
    # normalize meter_type
    df["meter_type"] = df["meter_type"].str.capitalize()
    return df


# This was the part of this challenge I found tricky.
 Pandas generates month starts that are on or after the start date.
 Since the contract start date (CSD) is 2019-07-20, the first month start is 2019-08-01, so July was dissapearing.
 So I had to floor the calendar at the first day of the month that contains the contract start date to achieve the output in the HTML

**Helper: `build_calendar`**
   - Floor the start date to the first of its month.  
   - Generate one row per calendar month through the contract end.  
   - Compute how many days of each month fall within the contract window.


In [4]:
def build_calendar(csd: str, ced: str) -> pd.DataFrame:
    first_month = pd.to_datetime(csd).replace(day=1)
    periods     = pd.date_range(start=first_month, end=ced, freq="MS")
    cal = pd.DataFrame({"period": periods})

    def days_in_contract(period):
        month_end = (period + pd.offsets.MonthEnd(0)).normalize()
        start = max(period, pd.to_datetime(csd))
        end   = min(month_end, pd.to_datetime(ced))
        return max((end - start).days + 1, 0)

    cal["no_days"] = cal["period"].map(days_in_contract)
    return cal


**Helper: `cross_join_mprns`**
- Create one row per MPRN per calendar month, carrying the `no_days` count.

In [5]:
def cross_join_mprns(calendar: pd.DataFrame, mprns: pd.DataFrame) -> pd.DataFrame:
    cal = calendar.copy(); cal["key"] = 1
    m   = mprns.copy();        m["key"]   = 1
    merged = cal.merge(m, on="key").drop("key", axis=1)
    return merged[["mprn", "meter_type", "meter_capacity", "period", "no_days"]]


**Helper: `load_map_costs`**  
- Read the MAP cost bands, parse the date windows, and normalize `meter_type`.

In [6]:
def load_map_costs(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, parse_dates=["from_date", "to_date"])
    df["meter_type"] = df["meter_type"].str.capitalize()
    return df


**Helper: `assign_price`**
   - Merge once on `meter_type`.  
   - Filter to bands where `meter_capacity` and `period` overlap the band’s date window.  
   - Assign `price_per_day`, leaving unmatched as `NaN`.

In [7]:
def assign_price(pricing_subset: pd.DataFrame, map_costs: pd.DataFrame) -> pd.DataFrame:
    merged = pricing_subset.merge(
        map_costs,
        on="meter_type",
        how="left",
        suffixes=("", "_map")
    )
    mask = (
        (merged["meter_capacity"] >= merged["min_capacity"]) &
        (merged["meter_capacity"] <  merged["max_capacity"]) &
        (merged["period"]         >= merged["from_date"]) &
        (merged["period"]         <= merged["to_date"])
    )
    merged["price_per_day"] = np.nan
    merged.loc[mask, "price_per_day"] = merged.loc[mask, "price"]
    return merged


**Helper: `compute_average_prices`**
   - Drop rows without a valid `price_per_day`.  
   - Compute `weighted = price_per_day × no_days`.  
   - Aggregate per MPRN, summing `no_days` and `weighted`.  
   - Compute `rate = (total_weighted ÷ total_days) × 100` to convert to pence/day.

In [8]:
def compute_average_pence(pricing: pd.DataFrame) -> pd.DataFrame:
    #  Only keep rows where we have a valid daily price
    valid = pricing.dropna(subset=["price_per_day"])

    # Compute weighted standing charge
    tmp = valid.assign(weighted = valid["price_per_day"] * valid["no_days"])

    # Aggregate per MPRN
    result = (
        tmp.groupby("mprn", as_index=False)
           .agg(
               no_days       = ("no_days",    "sum"),
               total_weighted= ("weighted",   "sum"),
           )
    )

    # Compute rate in pence/day
    result["rate"] = result["total_weighted"] / result["no_days"] * 100

    #  Return only the required columns
    return result[["mprn", "no_days", "rate"]]



**Main Execution Flow**  
   Orchestrate all steps: load inputs, build calendar, assign prices, compute rates, and output the final DataFrame.

In [ ]:
if __name__ == "__main__":
    #  Load inputs
    mprns     = load_and_preprocess_mprns(mprns_path)
    calendar  = build_calendar(CSD, CED)
    subset    = cross_join_mprns(calendar, mprns)
    map_costs = load_map_costs(map_costs_path)

    #  Assign prices and compute final rates
    priced  = assign_price(subset, map_costs)
    result = compute_average_pence(priced)


In [ ]:
result

,mprn,no_days,rate
0,47218110,83,379.656036
1,47218604,83,27.310348
